In [2]:
import time
import random
from colorsys import hsv_to_rgb
import board
from digitalio import DigitalInOut, Direction
from PIL import Image, ImageDraw, ImageFont
from adafruit_rgb_display import st7789
import numpy as np

# SPI 및 디스플레이 초기화
spi = board.SPI()
tft = st7789.ST7789(spi, rotation=0, width=240, height=240)

# 배경색 설정
color = 0x0000FF  # 파란색
fill = displayio.TileGrid(displayio.Bitmap(240, 240, 1), pixel_shader=displayio.ColorConverter())
fill[0] = color
group = displayio.Group()
group.append(fill)

# 디스플레이에 내용 표시
tft.show(group)
time.sleep(5)  # 화면 유지 시간


TypeError: ST7789.__init__() missing 2 required positional arguments: 'dc' and 'cs'